### Task 1: Querying MongoDB using PyMongo

1. Write a python program that will read the data from hotspot_historic.csv andclimate_historic.csv and load them to the new database(e.g.assignment_db).</br>- Please use a csv library to read the files.</br>- Please DO NOT use the mongo aggregation query to do this task.

In [1]:
# import the necessary library
import pymongo
from pymongo import MongoClient
import pandas as pd
from datetime import datetime
from pprint import pprint
print("Pandas Version: " + pd.__version__)

Pandas Version: 1.4.2


In [2]:
# create a MongoClient
client = MongoClient()
db = client.assignment_db

Create the 2 collections, climate_historic and hotspot_historic

In [3]:
# Drop the collections if they exist
db.climate_historic.drop()
db.hotspot_historic.drop()

# Create 2 collections for climate_historic and hotspot_historic
climate_historic = db.climate_historic
hotspot_historic = db.hotspot_historic

Reading the *climate_historic.csv* file using pandas library, format the data and insert it into the climate_historic collection

In [4]:
# read the climate_historic.csv file using pandas read_csv function
climate_historic_reader = pd.read_csv('climate_historic.csv')

# insert each climate historic data into the climate_historic_collection
for i in range(len(climate_historic_reader)):
    precipitation_data = climate_historic_reader["precipitation "][i].replace(" ", "")
    
    # prepare the climate data in the correct format and datatype
    climate_data = {
            "station": int(climate_historic_reader["station"][i]), 
            "date": datetime.strptime(climate_historic_reader["date"][i], "%d/%m/%y"), 
            "air_temperature_celcius": int(climate_historic_reader["air_temperature_celcius"][i]),
            "relative_humidity": float(climate_historic_reader["relative_humidity"][i]),
            "windspeed_knots": float(climate_historic_reader["windspeed_knots"][i]),
            "max_wind_speed": float(climate_historic_reader["max_wind_speed"][i]),
            "precipitation": {"amount": float(precipitation_data[-len(precipitation_data):-1]), "flag": precipitation_data[-1]},
            "GHI_w/m2": int(climate_historic_reader["GHI_w/m2"][i])
            
           }
    climate_historic.insert_one(climate_data)

Reading the *hotspot_historic.csv* file, format the data and insert it into the hotspot_historic collection

In [5]:
# read the hotspot_historic.csv file using pandas read_csv function
hotspot_historic_reader = pd.read_csv("hotspot_historic.csv")

# insert each hotspot historic data into the hotspot_historic_collection
for i in range(len(hotspot_historic_reader)):
    
    # prepare the hotspot data in the correct format and datatype
    hotspot_data = {
            "latitude": float(hotspot_historic_reader["latitude"][i]),
            "longitude": float(hotspot_historic_reader["longitude"][i]), 
            "datetime": datetime.strptime(hotspot_historic_reader["datetime"][i], "%Y-%m-%dT%H:%M:%S"),
            "confidence": int(hotspot_historic_reader["confidence"][i]),
            "date": datetime.strptime(hotspot_historic_reader["date"][i], '%d/%m/%Y'),
            "surface_temperature_celcius": int(hotspot_historic_reader["surface_temperature_celcius"][i])
           }
    hotspot_historic.insert_one(hotspot_data)

Shows that the database *assignment.db* has been created

In [6]:
for x in client.list_databases():
    print(x)

{'name': 'admin', 'sizeOnDisk': 40960.0, 'empty': False}
{'name': 'config', 'sizeOnDisk': 110592.0, 'empty': False}
{'name': 'fit3182_assignment_db', 'sizeOnDisk': 163840.0, 'empty': False}
{'name': 'fit3182_db', 'sizeOnDisk': 73728.0, 'empty': False}
{'name': 'local', 'sizeOnDisk': 81920.0, 'empty': False}


Shows the example of the climate data in the climate_historic collection

In [7]:
# sample of the document in climate_historic collection
climate_doc = climate_historic.find({}).limit(1)
for x in climate_doc: 
    pprint(x)

{'GHI_w/m2': 154,
 '_id': ObjectId('628a65b1cc5bb730f6ed6eaa'),
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2020, 12, 31, 0, 0),
 'max_wind_speed': 11.1,
 'precipitation': {'amount': 0.0, 'flag': 'I'},
 'relative_humidity': 56.8,
 'station': 948700,
 'windspeed_knots': 7.9}


Shows the example of the hotspot data in the hotspot_historic collection

In [8]:
# sample of the document in hotspot_historic collection
hotspot_doc = hotspot_historic.find({}).limit(1)
for x in hotspot_doc: 
    pprint(x)

{'_id': ObjectId('628a65b2cc5bb730f6ed7018'),
 'confidence': 78,
 'date': datetime.datetime(2021, 12, 27, 0, 0),
 'datetime': datetime.datetime(2021, 12, 27, 4, 16, 51),
 'latitude': -37.966,
 'longitude': 145.051,
 'surface_temperature_celcius': 68}


In [9]:
climate_historic.count_documents({})

366

In [10]:
hotspot_historic.count_documents({})

2668

2. Write queries to answer the following tasks on assignment_db and corresponding collection(s). You need to write the queries as a python program using the pymongo library in Jupyter Notebook.</br>

a. Find climate data on 12th December 2021.

In [11]:
climate_historic.find_one({"date": {'$eq': datetime(2021, 12, 12)}})

{'_id': ObjectId('628a65b1cc5bb730f6ed7003'),
 'station': 948702,
 'date': datetime.datetime(2021, 12, 12, 0, 0),
 'air_temperature_celcius': 19,
 'relative_humidity': 55.3,
 'windspeed_knots': 6.2,
 'max_wind_speed': 12.0,
 'precipitation': {'amount': 0.0, 'flag': 'I'},
 'GHI_w/m2': 156}

b. Find the latitude, longitude, surface temperature (°C), and confidence when
the surface temperature (°C) was between 65 °C and 100 °C.

In [12]:
query_b_results = hotspot_historic.find({"surface_temperature_celcius":{"$gte":65, "$lte":100}},
                                           {"_id":0,"latitude":1, "longitude":1, 
                                            "surface_temperature_celcius":1, "confidence":1})
for result in query_b_results:
    pprint(result)

{'confidence': 78,
 'latitude': -37.966,
 'longitude': 145.051,
 'surface_temperature_celcius': 68}
{'confidence': 86,
 'latitude': -35.543,
 'longitude': 143.316,
 'surface_temperature_celcius': 67}
{'confidence': 93,
 'latitude': -37.875,
 'longitude': 142.51,
 'surface_temperature_celcius': 73}
{'confidence': 95,
 'latitude': -37.613,
 'longitude': 149.305,
 'surface_temperature_celcius': 75}
{'confidence': 90,
 'latitude': -37.624,
 'longitude': 149.314,
 'surface_temperature_celcius': 66}
{'confidence': 93,
 'latitude': -38.057,
 'longitude': 144.211,
 'surface_temperature_celcius': 73}
{'confidence': 92,
 'latitude': -37.95,
 'longitude': 142.366,
 'surface_temperature_celcius': 70}
{'confidence': 100,
 'latitude': -36.282,
 'longitude': 146.157,
 'surface_temperature_celcius': 71}
{'confidence': 100,
 'latitude': -37.642,
 'longitude': 149.263,
 'surface_temperature_celcius': 65}
{'confidence': 100,
 'latitude': -37.634,
 'longitude': 149.237,
 'surface_temperature_celcius': 71}

 'surface_temperature_celcius': 88}
{'confidence': 100,
 'latitude': -35.6374,
 'longitude': 142.3787,
 'surface_temperature_celcius': 95}
{'confidence': 90,
 'latitude': -36.8835,
 'longitude': 142.2098,
 'surface_temperature_celcius': 67}
{'confidence': 100,
 'latitude': -36.4125,
 'longitude': 143.1189,
 'surface_temperature_celcius': 92}
{'confidence': 97,
 'latitude': -37.0669,
 'longitude': 141.0556,
 'surface_temperature_celcius': 80}
{'confidence': 92,
 'latitude': -36.0714,
 'longitude': 145.7665,
 'surface_temperature_celcius': 71}
{'confidence': 94,
 'latitude': -36.2933,
 'longitude': 141.36,
 'surface_temperature_celcius': 75}
{'confidence': 100,
 'latitude': -37.4591,
 'longitude': 144.9492,
 'surface_temperature_celcius': 89}
{'confidence': 91,
 'latitude': -36.3487,
 'longitude': 145.5315,
 'surface_temperature_celcius': 68}
{'confidence': 90,
 'latitude': -36.5399,
 'longitude': 144.678,
 'surface_temperature_celcius': 66}
{'confidence': 97,
 'latitude': -36.8158,
 'lo

c. Find date, surface temperature (°C), air temperature (°C), relative humidity and
max wind speed on 15th and 16th of December 2021.

1. First join the hotspot_historic collection with the climate_historic collection based on the common attribute "date" using "\\$lookup".
2. Second use "\\$project" to only show the required data for the output.
3. Last use "\\$match" to specifies the condition for the query.


In [13]:
query_c_results = hotspot_historic.aggregate([
    {
        "$lookup": {
            "from": "climate_historic",
            "localField": "date",
            "foreignField": "date",
            "as": "climate_data"
        }
    },
    
    {
        "$project": {
            "_id": 0,
            "date": 1,
            "surface_temperature_celcius": 1,
            "climate_data.air_temperature_celcius": 1,
            "climate_data.relative_humidity": 1,
            "climate_data.max_wind_speed": 1
        }
    },
    
    {
        "$match": {
            "$or":[{"date": {'$eq': datetime(2021, 12, 15, 0, 0)}}, 
                    {"date": {'$eq': datetime(2021, 12, 16, 0, 0)}}]
        }
    },
])
for result in query_c_results:
    pprint(result)

{'climate_data': [{'air_temperature_celcius': 18,
                   'max_wind_speed': 13.0,
                   'relative_humidity': 53.7}],
 'date': datetime.datetime(2021, 12, 16, 0, 0),
 'surface_temperature_celcius': 43}
{'climate_data': [{'air_temperature_celcius': 18,
                   'max_wind_speed': 13.0,
                   'relative_humidity': 53.7}],
 'date': datetime.datetime(2021, 12, 16, 0, 0),
 'surface_temperature_celcius': 33}
{'climate_data': [{'air_temperature_celcius': 18,
                   'max_wind_speed': 13.0,
                   'relative_humidity': 53.7}],
 'date': datetime.datetime(2021, 12, 16, 0, 0),
 'surface_temperature_celcius': 54}
{'climate_data': [{'air_temperature_celcius': 18,
                   'max_wind_speed': 13.0,
                   'relative_humidity': 53.7}],
 'date': datetime.datetime(2021, 12, 16, 0, 0),
 'surface_temperature_celcius': 73}
{'climate_data': [{'air_temperature_celcius': 18,
                   'max_wind_speed': 13.0,
       

d. Find datetime, air temperature (°C), surface temperature (°C) and confidence
when the confidence is between 80 and 100.

1. First join the hotspot_historic collection with the climate_historic collection based on the common attribute "date" using "\\$lookup".
2. Second use "\\$project" to only show the required data for the output.
3. Last use "\\$match" to specifies the condition for the query.


In [14]:
query_d_results = hotspot_historic.aggregate([
    {
        "$lookup": {
            "from": "climate_historic",
            "localField": "date",
            "foreignField": "date",
            "as": "climate_data"
        }
    },
    {
        "$project": {
            "_id": 0,
            "datetime": 1,
            "climate_data.air_temperature_celcius": 1,
            "surface_temperature_celcius": 1,
            "confidence": 1
        }
    },
    {
        "$match": {
            "$and": [{"confidence":{"$gte":80}}, {"confidence":{"$lte":100}}]}
    }
])
for document in query_d_results:
    pprint(document)

{'climate_data': [{'air_temperature_celcius': 28}],
 'confidence': 82,
 'datetime': datetime.datetime(2021, 12, 27, 0, 2, 15),
 'surface_temperature_celcius': 63}
{'climate_data': [{'air_temperature_celcius': 28}],
 'confidence': 86,
 'datetime': datetime.datetime(2021, 12, 27, 0, 2, 14),
 'surface_temperature_celcius': 67}
{'climate_data': [{'air_temperature_celcius': 17}],
 'confidence': 80,
 'datetime': datetime.datetime(2021, 12, 25, 4, 29, 8),
 'surface_temperature_celcius': 54}
{'climate_data': [{'air_temperature_celcius': 18}],
 'confidence': 94,
 'datetime': datetime.datetime(2021, 12, 16, 15, 38, 39),
 'surface_temperature_celcius': 43}
{'climate_data': [{'air_temperature_celcius': 18}],
 'confidence': 93,
 'datetime': datetime.datetime(2021, 12, 16, 4, 35, 13),
 'surface_temperature_celcius': 73}
{'climate_data': [{'air_temperature_celcius': 18}],
 'confidence': 84,
 'datetime': datetime.datetime(2021, 12, 16, 4, 34, 58),
 'surface_temperature_celcius': 55}
{'climate_data': [

{'climate_data': [{'air_temperature_celcius': 14}],
 'confidence': 100,
 'datetime': datetime.datetime(2021, 9, 24, 15, 7, 47),
 'surface_temperature_celcius': 65}
{'climate_data': [{'air_temperature_celcius': 14}],
 'confidence': 90,
 'datetime': datetime.datetime(2021, 9, 24, 15, 7, 47),
 'surface_temperature_celcius': 41}
{'climate_data': [{'air_temperature_celcius': 14}],
 'confidence': 99,
 'datetime': datetime.datetime(2021, 9, 24, 15, 7, 47),
 'surface_temperature_celcius': 61}
{'climate_data': [{'air_temperature_celcius': 14}],
 'confidence': 94,
 'datetime': datetime.datetime(2021, 9, 24, 15, 7, 45),
 'surface_temperature_celcius': 43}
{'climate_data': [{'air_temperature_celcius': 14}],
 'confidence': 100,
 'datetime': datetime.datetime(2021, 9, 24, 15, 7, 45),
 'surface_temperature_celcius': 61}
{'climate_data': [{'air_temperature_celcius': 14}],
 'confidence': 100,
 'datetime': datetime.datetime(2021, 9, 24, 13, 30, 10),
 'surface_temperature_celcius': 47}
{'climate_data': [

{'climate_data': [{'air_temperature_celcius': 20}],
 'confidence': 84,
 'datetime': datetime.datetime(2021, 4, 20, 4, 44, 20),
 'surface_temperature_celcius': 58}
{'climate_data': [{'air_temperature_celcius': 20}],
 'confidence': 93,
 'datetime': datetime.datetime(2021, 4, 20, 4, 34, 10),
 'surface_temperature_celcius': 72}
{'climate_data': [{'air_temperature_celcius': 20}],
 'confidence': 88,
 'datetime': datetime.datetime(2021, 4, 20, 4, 33, 20),
 'surface_temperature_celcius': 63}
{'climate_data': [{'air_temperature_celcius': 20}],
 'confidence': 90,
 'datetime': datetime.datetime(2021, 4, 20, 4, 32, 50),
 'surface_temperature_celcius': 67}
{'climate_data': [{'air_temperature_celcius': 20}],
 'confidence': 85,
 'datetime': datetime.datetime(2021, 4, 20, 4, 32, 30),
 'surface_temperature_celcius': 60}
{'climate_data': [{'air_temperature_celcius': 20}],
 'confidence': 80,
 'datetime': datetime.datetime(2021, 4, 20, 4, 32, 30),
 'surface_temperature_celcius': 53}
{'climate_data': [{'ai

 'surface_temperature_celcius': 93}
{'climate_data': [{'air_temperature_celcius': 16}],
 'confidence': 95,
 'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
 'surface_temperature_celcius': 77}
{'climate_data': [{'air_temperature_celcius': 16}],
 'confidence': 81,
 'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
 'surface_temperature_celcius': 54}
{'climate_data': [{'air_temperature_celcius': 16}],
 'confidence': 95,
 'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
 'surface_temperature_celcius': 75}
{'climate_data': [{'air_temperature_celcius': 16}],
 'confidence': 84,
 'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
 'surface_temperature_celcius': 58}
{'climate_data': [{'air_temperature_celcius': 16}],
 'confidence': 84,
 'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
 'surface_temperature_celcius': 58}
{'climate_data': [{'air_temperature_celcius': 16}],
 'confidence': 86,
 'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
 'surface_temperature_

e. Find the top 10 records with the highest surface temperature (°C).

In [15]:
# sort the hotspot_historic data descendingly based on the surface_temperature_celcius and get the top 10
query_e_results = hotspot_historic.find().sort("surface_temperature_celcius", -1).limit(10)
for document in query_e_results:
    pprint(document)

{'_id': ObjectId('628a65b3cc5bb730f6ed74bb'),
 'confidence': 100,
 'date': datetime.datetime(2021, 4, 18, 0, 0),
 'datetime': datetime.datetime(2021, 4, 18, 4, 52),
 'latitude': -38.1665,
 'longitude': 143.062,
 'surface_temperature_celcius': 124}
{'_id': ObjectId('628a65b3cc5bb730f6ed7965'),
 'confidence': 100,
 'date': datetime.datetime(2021, 4, 4, 0, 0),
 'datetime': datetime.datetime(2021, 4, 4, 4, 32, 50),
 'latitude': -36.343,
 'longitude': 142.1986,
 'surface_temperature_celcius': 123}
{'_id': ObjectId('628a65b3cc5bb730f6ed742d'),
 'confidence': 100,
 'date': datetime.datetime(2021, 5, 1, 0, 0),
 'datetime': datetime.datetime(2021, 5, 1, 4, 14, 20),
 'latitude': -36.9318,
 'longitude': 143.0907,
 'surface_temperature_celcius': 122}
{'_id': ObjectId('628a65b3cc5bb730f6ed7a54'),
 'confidence': 100,
 'date': datetime.datetime(2021, 3, 18, 0, 0),
 'datetime': datetime.datetime(2021, 3, 18, 3, 50, 50),
 'latitude': -37.017,
 'longitude': 148.1297,
 'surface_temperature_celcius': 121}

f. Find the number of fires each day. You are required to only display the total
number of fires and the date in the output.

In [16]:
query_f_results = hotspot_historic.aggregate([{"$group":{"_id":"$date", "count":{"$sum":1}}}])
for document in query_f_results:
    pprint(document)

{'_id': datetime.datetime(2021, 12, 27, 0, 0), 'count': 4}
{'_id': datetime.datetime(2021, 7, 14, 0, 0), 'count': 2}
{'_id': datetime.datetime(2021, 7, 5, 0, 0), 'count': 1}
{'_id': datetime.datetime(2021, 4, 1, 0, 0), 'count': 7}
{'_id': datetime.datetime(2021, 9, 21, 0, 0), 'count': 2}
{'_id': datetime.datetime(2021, 8, 5, 0, 0), 'count': 1}
{'_id': datetime.datetime(2021, 4, 23, 0, 0), 'count': 19}
{'_id': datetime.datetime(2021, 3, 7, 0, 0), 'count': 1}
{'_id': datetime.datetime(2021, 6, 18, 0, 0), 'count': 2}
{'_id': datetime.datetime(2021, 6, 13, 0, 0), 'count': 1}
{'_id': datetime.datetime(2021, 10, 28, 0, 0), 'count': 1}
{'_id': datetime.datetime(2021, 11, 8, 0, 0), 'count': 2}
{'_id': datetime.datetime(2021, 10, 21, 0, 0), 'count': 4}
{'_id': datetime.datetime(2021, 12, 24, 0, 0), 'count': 1}
{'_id': datetime.datetime(2021, 10, 3, 0, 0), 'count': 18}
{'_id': datetime.datetime(2021, 5, 11, 0, 0), 'count': 19}
{'_id': datetime.datetime(2021, 5, 3, 0, 0), 'count': 64}
{'_id': dat

g. Find the records of fires where the confidence is below 70.

In [17]:
results = hotspot_historic.find({"confidence":{"$lt":70}})
for result in results:
    pprint(result)

{'_id': ObjectId('628a65b2cc5bb730f6ed701a'),
 'confidence': 67,
 'date': datetime.datetime(2021, 12, 27, 0, 0),
 'datetime': datetime.datetime(2021, 12, 27, 0, 2, 15),
 'latitude': -35.554,
 'longitude': 143.307,
 'surface_temperature_celcius': 53}
{'_id': ObjectId('628a65b2cc5bb730f6ed701d'),
 'confidence': 65,
 'date': datetime.datetime(2021, 12, 24, 0, 0),
 'datetime': datetime.datetime(2021, 12, 24, 13, 12, 1),
 'latitude': -35.646,
 'longitude': 142.282,
 'surface_temperature_celcius': 32}
{'_id': ObjectId('628a65b2cc5bb730f6ed7020'),
 'confidence': 69,
 'date': datetime.datetime(2021, 12, 16, 0, 0),
 'datetime': datetime.datetime(2021, 12, 16, 15, 38, 39),
 'latitude': -37.624,
 'longitude': 149.332,
 'surface_temperature_celcius': 33}
{'_id': ObjectId('628a65b2cc5bb730f6ed7021'),
 'confidence': 62,
 'date': datetime.datetime(2021, 12, 16, 0, 0),
 'datetime': datetime.datetime(2021, 12, 16, 4, 35, 15),
 'latitude': -37.82,
 'longitude': 142.323,
 'surface_temperature_celcius': 5

 'longitude': 143.2041,
 'surface_temperature_celcius': 39}
{'_id': ObjectId('628a65b2cc5bb730f6ed727f'),
 'confidence': 54,
 'date': datetime.datetime(2021, 5, 11, 0, 0),
 'datetime': datetime.datetime(2021, 5, 11, 4, 50, 40),
 'latitude': -36.3909,
 'longitude': 141.1843,
 'surface_temperature_celcius': 39}
{'_id': ObjectId('628a65b2cc5bb730f6ed7285'),
 'confidence': 58,
 'date': datetime.datetime(2021, 5, 11, 0, 0),
 'datetime': datetime.datetime(2021, 5, 11, 4, 50, 40),
 'latitude': -36.4713,
 'longitude': 144.4747,
 'surface_temperature_celcius': 40}
{'_id': ObjectId('628a65b2cc5bb730f6ed7287'),
 'confidence': 68,
 'date': datetime.datetime(2021, 5, 11, 0, 0),
 'datetime': datetime.datetime(2021, 5, 11, 0, 33, 20),
 'latitude': -38.5675,
 'longitude': 146.4563,
 'surface_temperature_celcius': 44}
{'_id': ObjectId('628a65b2cc5bb730f6ed728c'),
 'confidence': 66,
 'date': datetime.datetime(2021, 5, 10, 0, 0),
 'datetime': datetime.datetime(2021, 5, 10, 4, 14, 30),
 'latitude': -36.79

 'datetime': datetime.datetime(2021, 4, 18, 4, 44, 50),
 'latitude': -36.9986,
 'longitude': 143.8953,
 'surface_temperature_celcius': 44}
{'_id': ObjectId('628a65b3cc5bb730f6ed758c'),
 'confidence': 52,
 'date': datetime.datetime(2021, 4, 18, 0, 0),
 'datetime': datetime.datetime(2021, 4, 18, 4, 44, 50),
 'latitude': -36.742,
 'longitude': 144.8955,
 'surface_temperature_celcius': 38}
{'_id': ObjectId('628a65b3cc5bb730f6ed758d'),
 'confidence': 66,
 'date': datetime.datetime(2021, 4, 18, 0, 0),
 'datetime': datetime.datetime(2021, 4, 18, 4, 44, 50),
 'latitude': -36.9285,
 'longitude': 143.9622,
 'surface_temperature_celcius': 43}
{'_id': ObjectId('628a65b3cc5bb730f6ed758f'),
 'confidence': 54,
 'date': datetime.datetime(2021, 4, 18, 0, 0),
 'datetime': datetime.datetime(2021, 4, 18, 4, 44, 50),
 'latitude': -36.9139,
 'longitude': 143.9765,
 'surface_temperature_celcius': 42}
{'_id': ObjectId('628a65b3cc5bb730f6ed7593'),
 'confidence': 65,
 'date': datetime.datetime(2021, 4, 18, 0, 0

{'_id': ObjectId('628a65b3cc5bb730f6ed78c1'),
 'confidence': 59,
 'date': datetime.datetime(2021, 4, 6, 0, 0),
 'datetime': datetime.datetime(2021, 4, 6, 4, 20, 40),
 'latitude': -37.6806,
 'longitude': 143.2676,
 'surface_temperature_celcius': 51}
{'_id': ObjectId('628a65b3cc5bb730f6ed78c9'),
 'confidence': 58,
 'date': datetime.datetime(2021, 4, 6, 0, 0),
 'datetime': datetime.datetime(2021, 4, 6, 4, 20, 40),
 'latitude': -37.7212,
 'longitude': 145.9754,
 'surface_temperature_celcius': 52}
{'_id': ObjectId('628a65b3cc5bb730f6ed78ca'),
 'confidence': 59,
 'date': datetime.datetime(2021, 4, 6, 0, 0),
 'datetime': datetime.datetime(2021, 4, 6, 4, 20, 40),
 'latitude': -38.3343,
 'longitude': 143.8644,
 'surface_temperature_celcius': 43}
{'_id': ObjectId('628a65b3cc5bb730f6ed78d0'),
 'confidence': 64,
 'date': datetime.datetime(2021, 4, 6, 0, 0),
 'datetime': datetime.datetime(2021, 4, 6, 4, 20, 40),
 'latitude': -35.723,
 'longitude': 142.063,
 'surface_temperature_celcius': 54}
{'_id'

h. Find the average surface temperature (°C) for each day. You are required to
only display average surface temperature (°C) and the date in the output.

In [18]:
query_h_results = hotspot_historic.aggregate([
    {
        "$group":
        {
            "_id":"$date", 
            "avgSurfaceTemperature":{"$avg":"$surface_temperature_celcius"}
         }
    }])
for document in query_h_results:
    pprint(document)

{'_id': datetime.datetime(2021, 12, 27, 0, 0), 'avgSurfaceTemperature': 62.75}
{'_id': datetime.datetime(2021, 7, 14, 0, 0), 'avgSurfaceTemperature': 41.5}
{'_id': datetime.datetime(2021, 7, 5, 0, 0), 'avgSurfaceTemperature': 45.0}
{'_id': datetime.datetime(2021, 4, 1, 0, 0),
 'avgSurfaceTemperature': 46.714285714285715}
{'_id': datetime.datetime(2021, 9, 21, 0, 0), 'avgSurfaceTemperature': 40.5}
{'_id': datetime.datetime(2021, 8, 5, 0, 0), 'avgSurfaceTemperature': 40.0}
{'_id': datetime.datetime(2021, 4, 23, 0, 0),
 'avgSurfaceTemperature': 53.89473684210526}
{'_id': datetime.datetime(2021, 3, 7, 0, 0), 'avgSurfaceTemperature': 64.0}
{'_id': datetime.datetime(2021, 6, 18, 0, 0), 'avgSurfaceTemperature': 42.0}
{'_id': datetime.datetime(2021, 6, 13, 0, 0), 'avgSurfaceTemperature': 41.0}
{'_id': datetime.datetime(2021, 10, 28, 0, 0), 'avgSurfaceTemperature': 56.0}
{'_id': datetime.datetime(2021, 11, 8, 0, 0), 'avgSurfaceTemperature': 45.5}
{'_id': datetime.datetime(2021, 10, 21, 0, 0), '

i. Find the top 10 records with the lowest GHI.

In [19]:
# sort the hotspot_historic data ascendingly based on GHI_w/m2 and get the top 10
query_i_results = climate_historic.find().sort("GHI_w/m2").limit(10)
for document in query_i_results:
    pprint(document)

{'GHI_w/m2': 47,
 '_id': ObjectId('628a65b1cc5bb730f6ed6f7f'),
 'air_temperature_celcius': 5,
 'date': datetime.datetime(2021, 8, 2, 0, 0),
 'max_wind_speed': 5.1,
 'precipitation': {'amount': 0.0, 'flag': 'I'},
 'relative_humidity': 38.6,
 'station': 948701,
 'windspeed_knots': 1.8}
{'GHI_w/m2': 48,
 '_id': ObjectId('628a65b1cc5bb730f6ed6f5e'),
 'air_temperature_celcius': 5,
 'date': datetime.datetime(2021, 6, 30, 0, 0),
 'max_wind_speed': 11.1,
 'precipitation': {'amount': 0.0, 'flag': 'A'},
 'relative_humidity': 34.9,
 'station': 948701,
 'windspeed_knots': 4.3}
{'GHI_w/m2': 49,
 '_id': ObjectId('628a65b1cc5bb730f6ed6f5f'),
 'air_temperature_celcius': 5,
 'date': datetime.datetime(2021, 7, 1, 0, 0),
 'max_wind_speed': 14.0,
 'precipitation': {'amount': 0.0, 'flag': 'I'},
 'relative_humidity': 33.1,
 'station': 948701,
 'windspeed_knots': 5.8}
{'GHI_w/m2': 56,
 '_id': ObjectId('628a65b1cc5bb730f6ed6f69'),
 'air_temperature_celcius': 6,
 'date': datetime.datetime(2021, 7, 11, 0, 0),
 

j. Find the records with a 24-hour precipitation recorded between 0.20 to 0.35.

In [20]:
query_j_results = climate_historic.find({"precipitation.flag":{"$eq":'G'}, 
                                         "$and":[{"precipitation.amount":{"$gte":0.20}}, 
                                                 {"precipitation.amount":{"$lte":0.35}}]})

for result in query_j_results:
    pprint(result)

{'GHI_w/m2': 157,
 '_id': ObjectId('628a65b1cc5bb730f6ed6eb6'),
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2021, 1, 13, 0, 0),
 'max_wind_speed': 18.1,
 'precipitation': {'amount': 0.31, 'flag': 'G'},
 'relative_humidity': 54.1,
 'station': 948700,
 'windspeed_knots': 11.2}
{'GHI_w/m2': 146,
 '_id': ObjectId('628a65b1cc5bb730f6ed6f01'),
 'air_temperature_celcius': 17,
 'date': datetime.datetime(2021, 3, 29, 0, 0),
 'max_wind_speed': 21.0,
 'precipitation': {'amount': 0.24, 'flag': 'G'},
 'relative_humidity': 49.9,
 'station': 948701,
 'windspeed_knots': 12.2}
{'GHI_w/m2': 166,
 '_id': ObjectId('628a65b1cc5bb730f6ed6f17'),
 'air_temperature_celcius': 20,
 'date': datetime.datetime(2021, 4, 20, 0, 0),
 'max_wind_speed': 15.9,
 'precipitation': {'amount': 0.31, 'flag': 'G'},
 'relative_humidity': 53.5,
 'station': 948701,
 'windspeed_knots': 7.2}
{'GHI_w/m2': 102,
 '_id': ObjectId('628a65b1cc5bb730f6ed6f1d'),
 'air_temperature_celcius': 11,
 'date': datetime.datetime(2021,

#### Reference
- Converting string to datetime refer from: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
